### 20year는 데이터가 많아서 3번 이상 등장으로 변경

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import copy
import json
import random
import io
import distutils.dir_util
import warnings
warnings.filterwarnings("ignore")
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity

import os, sys, gc, time, warnings, pickle, psutil, random
import gc
import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

        
from collections import Counter
from tqdm import tqdm



In [ ]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(parent)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [ ]:
train = pd.read_parquet('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/train(song_trim).parquet')
val= pd.read_json('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/val(lower).json')


In [ ]:
# train = pd.read_json('/kaggle/input/kakao-arena-melon/train.json')
# # train_df = pd.read_json('/kaggle/input/kakao-train-valid-split/train_df.json')
# val = pd.read_json('/kaggle/input/kakao-arena-melon/val.json')
# # song_meta = pd.read_json('/kaggle/input/kakao-arena-melon/song_meta.json')
# # val_x = pd.read_json('/kaggle/input/kakao-train-valid-split/val_x.json')
# # val_y = pd.read_json('/kaggle/input/kakao-train-valid-split/val_y.json')
# # genre_gn_all = pd.read_json('/kaggle/input/kakao-arena-melon/genre_gn_all.json', typ = 'series')

In [ ]:
def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists[col]:
        c.update(doc)

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

def remove_seen(seen, l):
    seen = set(seen)
    ## 데이터에 l의 원소가 없으면 l의 원소를 채워 넣는다. 
    return [x for x in l if not (x in seen)]

### popular_yaer

In [ ]:
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))


## 데이터 만들기


In [ ]:
def before_pivot(data,use_song):
    result = []
    start = time.time()  # 시작 시간 저장     

    for i in tqdm(range(len(data))):
        for song in data.loc[i:i, 'songs'].values[0]:
            if song in use_song:
                df = pd.DataFrame({
                            'id' : [data.loc[i:i, 'id'].values[0]],
                            'song' : [song],
                            'point' : 1
                        })
                result += [df]

    final_result = pd.concat(result)
    print(final_result.shape)
    end = time.time()  # 시작 시간 저장     
    print("spend time ", round((end-start)/60, 2))
    return final_result

In [ ]:
def make_use_song(train_tmp):
    c = Counter()

    for doc in train_tmp['songs']:
        c.update(doc)


    tmp_dict= pd.DataFrame.from_dict(c, orient = 'index')
    print("before ", tmp_dict.shape)
    tmp_dict = tmp_dict[tmp_dict[0] > 2]
    print("after ", tmp_dict.shape)
    tmp_dict.reset_index(inplace = True)

    ## 3회 이상 등장한 노래만 사용! ==> use_song에 list형태로 담음
    use_song = list(tmp_dict['index'])
    
    del c, tmp_dict
    gc.collect()
    return use_song


In [ ]:
def make_pv_file(small_years):
    ## TRAIN
    print("train only year", train[(train['year'].isin(small_years))].shape)
    train_tmp = train[(train['year'].isin(small_years))]


    train_tmp = train_tmp[(train_tmp['like_cnt'] > 0)]
    train_tmp.reset_index(inplace = True, drop = True)
    print("train best playlist", train_tmp.shape)
    
    use_song = make_use_song(train_tmp)
    


    train_df = before_pivot(train_tmp,use_song)
    print('complete train')
    # val_df = before_pivot(val_tmp)

    del train_tmp 
    gc.collect()


    # all_df = pd.concat([train_df, val_df])
    train_df = train_df.pivot(index = 'song', columns ='id', values = 'point').fillna(0)
    train_df.columns = train_df.columns.astype(str)
    print("최종 shape : ", train_df.shape)

    train_df.to_parquet('train_DF_%s.parquet' %small_years)
    del train_df
    gc.collect()


In [ ]:
year_split = [[20]]

In [ ]:
for small_years in year_split:
    make_pv_file(small_years)